# Import

In [5]:
# ============================================
# CORE PYTHON LIBRARIES
# ============================================
import os                            # For file and directory operations
from pathlib import Path             # For platform-independent file paths
import glob                          # File pattern matching

# ============================================
# NUMERICAL & IMAGE PROCESSING
# ============================================
import cv2                           # OpenCV for image and video processing
import numpy as np                   # Core numerical computing library
from PIL import Image                # Python Image Library

# ============================================
# VISUALIZATION & ANALYSIS
# ============================================
import matplotlib.pyplot as plt       # General plotting and data visualization
import seaborn as sns                 # Statistical data visualization built on matplotlib
from tqdm.notebook import tqdm        # Progress bars for loops

# ============================================
# PYTORCH CORE LIBRARIES
# ============================================
import torch                          # Main PyTorch library
import torch.nn as nn                 # Neural network components
import torch.optim as optim           # Optimizers (Adam, SGD, etc.)
from torch.utils.data import DataLoader, Dataset  # Dataset utilities
from torch.optim.lr_scheduler import StepLR, ExponentialLR, CosineAnnealingLR  #LR schedulers

# ============================================
# TORCHVISION & AUGMENTATIONS
# ============================================
import albumentations                 # Advanced image augmentation library
from albumentations.pytorch import ToTensorV2  # Convert Albumentations outputs to tensors
import torchvision.transforms as T

# ============================================
# SEGMENTATION MODELS
# ============================================
import segmentation_models_pytorch as smp  # Pretrained segmentation architectures

import random
import pandas as pd
import time

import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt


# Hyperparameters

In [6]:
# define device
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

device = get_device()
print(device)

mps


In [7]:
# ===============================
# CONFIGURATION
# ===============================

config = {
    "batch_size": 10,          # Training batch size
    "test_batch_size": 1,    # Batch size for testing
    "epochs": 100,              # Number of training epochs
    "learning_rate": 0.001,    # Optimizer learning rate
    "momentum": 0.9,           # Momentum (if using SGD)
    "device": device,          # Automatically use GPU if available
    "random_seed": 42,         # For reproducibility
    "workers": 0,
}

#Directorys for data
root_directory = os.path.join("Kvasir-SEG")
images_directory = os.path.join(root_directory, "images")
masks_directory = os.path.join(root_directory, "masks")
validate_txt = ("val.txt")
train_txt = ("train.txt")

# Set random seed for reproducibility
torch.manual_seed(config["random_seed"])
np.random.seed(config["random_seed"])

#Set run name for model name and graph and data name
run_name = "TestRun"

if config["device"].type == 'cuda':
    torch.cuda.manual_seed(config["random_seed"])

# Check what version of PyTorch we have and what device we are using
print("PyTorch version:", torch.__version__)
print(f"Using device: {config['device']}")

PyTorch version: 2.8.0
Using device: mps


# Data import

In [10]:
def load_names(txt_path, ext):
    return [f"{ln.strip()}{ext}" for ln in Path(txt_path).read_text().splitlines() if ln.strip()]

val_imgs   = load_names(validate_txt,   ext=".jpg")
val_masks  = load_names(validate_txt,   ext=".jpg")
train_imgs = load_names(train_txt, ext=".jpg")
train_masks= load_names(train_txt, ext=".jpg")

# Data Explorer

In [11]:
def preprocess_mask(mask):
    return (mask > 127).astype(np.float32)

# Data Vizualiser

In [12]:
### My Fixed Verison from albumantation
def display_image_grid(images_filenames, images_directory, masks_directory, predicted_masks=None):
    rows = 3 if predicted_masks else 2
    cols = len(images_filenames)
    _, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(24, 10))
    for i, image_filename in enumerate(images_filenames):
        # Fixed: cv2.IMREAD_COLOR_RGB doesn't exist, use cv2.IMREAD_COLOR and convert
        image = cv2.imread(os.path.join(images_directory, image_filename), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        #fixed line
        mask = cv2.imread(os.path.join(masks_directory, image_filename), cv2.IMREAD_GRAYSCALE)

        mask = preprocess_mask(mask)
        ax[0, i].imshow(image)
        ax[1, i].imshow(mask, interpolation="nearest", cmap="gray", vmin=0, vmax=1)

        ax[0, i].set_title("Image")
        ax[1, i].set_title("Ground truth mask")

        ax[0, i].set_axis_off()
        ax[1, i].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[2, i].imshow(predicted_mask, interpolation="nearest", cmap="gray", vmin=0, vmax=1)
            ax[2, i].set_title("Predicted mask")
            ax[2, i].set_axis_off()
    plt.tight_layout()
    plt.show()

# Data Loader

In [13]:
class Kvasir_Dataset(Dataset):
    def __init__(self, images_names, mask_names, images_directory, masks_directory, transform=None):
        self.images_names = images_names
        self.mask_names = mask_names
        self.images_directory = images_directory
        self.masks_directory = masks_directory
        self.transform = transform

    def __len__(self):
        return len(self.images_names)

    def __getitem__(self, idx):
        image_filename = self.images_names[idx]
        image = cv2.imread(os.path.join(self.images_directory, image_filename))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask_filename = self.mask_names[idx]
        mask = cv2.imread(
            os.path.join(self.masks_directory, mask_filename),
            cv2.IMREAD_GRAYSCALE
        )

        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]

        if mask.dtype != torch.float32:
            mask = mask.float()
        # ToTensorV2 made mask 0..255 as float; scale then threshold:
        mask = (mask / 255.0 > 0.5).float()            # 0/1
        if mask.ndim == 2:
            mask = mask.unsqueeze(0)                   # 1xHxW

        return image, mask

In [14]:
def loaders(config, train_ds, val_ds):
    if config["workers"] > 0:
        trainloader = torch.utils.data.DataLoader(
            train_ds,
            batch_size=config["batch_size"],
            shuffle=True,
            num_workers=config["workers"],
            prefetch_factor=2,
            persistent_workers=True,
            drop_last=True,
        )
        
        testloader = torch.utils.data.DataLoader(
            val_ds,
            batch_size=config["test_batch_size"],
            num_workers=config["workers"],
            prefetch_factor=2,
            persistent_workers=True,
        )
    else:
        # Single-threaded mode (safer for different systems)
        trainloader = torch.utils.data.DataLoader(
            train_ds,
            batch_size=config["batch_size"],
            shuffle=True,
            num_workers=0,
            drop_last=True,
        )
        
        testloader = torch.utils.data.DataLoader(
            val_ds,
            batch_size=config["test_batch_size"],
            num_workers=0,
        )

    return trainloader, testloader

# Checkpoint and return functions

In [15]:
def get_models_dir():
    # Use __file__ if available, otherwise fallback to current working directory
    base_dir = os.path.dirname(__file__) if "__file__" in globals() else os.getcwd()
    models_dir = os.path.join(base_dir, "Models")
    os.makedirs(models_dir, exist_ok=True)
    return models_dir

def checkpoint(model, filename,path):
    models_dir = get_models_dir()
    filepath = os.path.join(models_dir, filename)
    torch.save(model.state_dict(), filepath)
    print(f"Model checkpoint saved to: {filepath}")

def resume(model, filename):
    models_dir = get_models_dir()
    filepath = os.path.join(models_dir, filename)

    if not os.path.exists(filepath):
        raise FileNotFoundError(f"No checkpoint found at: {filepath}")

    model.load_state_dict(torch.load(filepath))
    print(f"Model checkpoint loaded from: {filepath}")


# Loss

In [16]:
EPS = 1e-7

def soft_dice_loss(logits, targets, eps=EPS):
    """
    logits: (N, 1, H, W) or (N, H, W)
    targets: same shape, float {0,1}
    """
    probs = torch.sigmoid(logits)

    if probs.ndim == 4:   # (N,1,H,W) -> (N, H*W)
        probs   = probs.view(probs.size(0), -1)
        targets = targets.view(targets.size(0), -1)
    elif probs.ndim == 3: # (N,H,W) -> (N, H*W)
        probs   = probs.view(probs.size(0), -1)
        targets = targets.view(targets.size(0), -1)
    else:
        raise ValueError(f"Unexpected shape: {logits.shape}")

    inter = (probs * targets).sum(dim=1)
    denom = probs.sum(dim=1) + targets.sum(dim=1)
    dice  = (2 * inter + eps) / (denom + eps)
    return 1.0 - dice.mean()   # loss (scalar)

In [17]:
def iou_loss(logits, targets, eps=EPS):
    preds = logits.sigmoid()
    preds = (preds > 0.5).float()
    inter = (preds * targets).sum(dim=(1,2,3))
    union = preds.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3)) - inter
    iou = (inter + eps) / (union + eps)
    return 1-iou.mean() # loss (scalar)

In [18]:
def precision_recall(logits, targets, eps=1e-7):
    preds = logits.sigmoid()
    preds = (preds > 0.5).float()
    tp = (preds * targets).sum()
    fp = (preds * (1 - targets)).sum()
    fn = ((1 - preds) * targets).sum()

    precision = (tp + eps) / (tp + fp + eps)
    recall    = (tp + eps) / (tp + fn + eps)
    return precision.item(), recall.item()

# Early Stopping

In [28]:
class EarlyStopping:
    def __init__(self, patience=2, delta=1e-3):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.best_model_state = None

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model_state = model.state_dict()
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_state = model.state_dict()
            self.counter = 0

In [29]:
early_stopper = EarlyStopping()

# Model

In [13]:
# Create model - easily switch between different architectures!
def create_model(architecture="deeplabv3plus", encoder="resnet34"):
    common_params = {
        "encoder_name": encoder,
        "encoder_weights": "imagenet",
        "in_channels": 3,
        "classes": 1,
    }

    if architecture == "unet":
        model = smp.Unet(**common_params)
    elif architecture == "unetplusplus":
        model = smp.UnetPlusPlus(**common_params)
    elif architecture == "deeplabv3plus":
        model = smp.DeepLabV3Plus(**common_params)
    elif architecture == "manet":
        model = smp.MAnet(**common_params)
    elif architecture == "linknet":
        model = smp.Linknet(**common_params)
    elif architecture == "fpn":
        model = smp.FPN(**common_params)
    elif architecture == "pspnet":
        model = smp.PSPNet(**common_params)
    else:
        raise ValueError(f"Unsupported architecture: {architecture}")
    model.to(config["device"])
    return model

# Train/Test/Validation Loop

In [31]:
def train(trainloader, testloader, model, criterion, optimizer, device, config, path, run_type):
    lambda_dice = 0.7
    lambda_bce = 0.4
    coef_to_beat = 0.0

    traininglogs = {"time":[], "lambda_bce": lambda_bce, "lambda_dice": lambda_dice, "epoch": [], "total_loss": [],"activation_func_loss": [],"dice_loss": [],"dice_coef": [],"Iou_loss":[],"Iou_score":[],"precision": [],"recall": [],"lr": []}
    testinglogs = {"time":[], "lambda_bce": lambda_bce, "lambda_dice": lambda_dice, "epoch": [], "total_loss": [],"activation_func_loss": [],"dice_loss": [],"dice_coef": [],"Iou_loss":[],"Iou_score":[],"precision": [],"recall": [],"lr": []}
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold=1e-3)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.5)
    epoch_bar = tqdm(range(config["epochs"]), desc="Epochs", leave=True)
    #print(f'λ_bce={lambda_bce}, λ_dice={lambda_dice}')
    for epoch in epoch_bar:  # loop over the dataset multiple times
        total_loss_sum = 0.0
        bce_loss_sum   = 0.0
        dice_loss_sum  = 0.0
        n_train        = 0
        model.train()
        train_loss_sum= 0.0
        IOU_loss_sum = 0.0
        precision_sum = 0.0
        recall_sum = 0.0

        train_bar = tqdm(trainloader, desc=f"Train {epoch+1}/{config['epochs']}", leave=False)
        start = time.time()
        for images, gt_masks in train_bar:
            #images = images.to(device)
            #gt_masks = gt_masks.to(device)
            #gt_masks = gt_masks.float().to(device)/255.0
            images   = images.to(device=device, dtype=torch.float32, non_blocking=True)
            gt_masks = gt_masks.to(device=device, dtype=torch.float32, non_blocking=True)
            


            logits = model(images)          # (N,H,W)
            loss_bce  = criterion(logits, gt_masks)
            loss_dice = soft_dice_loss(logits, gt_masks)
            loss = loss_bce * lambda_bce + lambda_dice * loss_dice

            IOU_loss = iou_loss(logits, gt_masks)
            precision, recall = precision_recall(logits, gt_masks)
            

        
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

            batch_sz = images.size(0)
            train_loss_sum += loss.item() * batch_sz
            bce_loss_sum += loss_bce.item() * batch_sz
            dice_loss_sum += loss_dice.item() * batch_sz
            IOU_loss_sum += IOU_loss.item() * batch_sz
            precision_sum += precision * batch_sz
            recall_sum += recall * batch_sz
            n_train += batch_sz

            train_bar.set_postfix({'Train Loss': train_loss_sum / max(1, n_train), 'Train BCE Loss': bce_loss_sum / max(1, n_train), 'Train Dice Loss': dice_loss_sum / max(1, n_train), 'Train dice coef': 1 - (dice_loss_sum / max(1, n_train)), 'Train IOU Loss': IOU_loss_sum / max(1, n_train), 'Train IOU Score': 1 - (IOU_loss_sum / max(1, n_train)), 'Precision': precision_sum / max(1, n_train), 'Recall': recall_sum / max(1, n_train)})
        end = time.time()
        train_epoch_time = end - start
        training_loss = train_loss_sum / max(1, n_train)
        training_dice_loss = dice_loss_sum / max(1, n_train)
        training_bce_loss = bce_loss_sum / max(1, n_train)
        avg_IOU_loss = IOU_loss_sum / max(1, n_train)
        lr = optimizer.param_groups[0]["lr"]
        avg_precision = precision_sum / max(1, n_train)
        avg_recall = recall_sum / max(1, n_train)

        ### logging ###
        traininglogs["time"].append(train_epoch_time)
        traininglogs["epoch"].append(epoch)
        traininglogs["total_loss"].append(training_loss)
        traininglogs["activation_func_loss"].append(training_bce_loss)
        traininglogs["dice_loss"].append(training_dice_loss)
        traininglogs["dice_coef"].append(1 - training_dice_loss)
        traininglogs["Iou_loss"].append(avg_IOU_loss)
        traininglogs["Iou_score"].append(1 - avg_IOU_loss)
        traininglogs["precision"].append(avg_precision)
        traininglogs["recall"].append(avg_recall)
        traininglogs["lr"].append(lr)


        print(f'Train Epoch [{epoch+1}/{config["epochs"]}] - Loss: {training_loss:.4f}, BCE Loss: {training_bce_loss:.4f}, Dice Loss: {training_dice_loss:.4f}, Dice Coef: {1 - training_dice_loss:.4f}, IOU Loss: {avg_IOU_loss:.4f}, IOU Score: {1 - avg_IOU_loss:.4f}, Precision: {avg_precision:.4f}, Recall: {avg_recall:.4f}')

        model.eval()
        n_val = 0

        with torch.no_grad():
            test_total_loss_sum = 0.0
            test_bce_loss_sum   = 0.0
            test_dice_loss_sum  = 0.0
            IOU_loss_sum = 0.0
            val_bar = tqdm(testloader, desc=f"Val   {epoch+1}/{config['epochs']}", leave=False)
            precision_sum = 0.0
            recall_sum = 0.0


            start = time.time()
            for images, gt_masks in val_bar:
                #gt_masks = gt_masks.to(device)/255.0
                #images = images.to(device)
                images   = images.to(device=device, dtype=torch.float32, non_blocking=True)
                gt_masks = gt_masks.to(device=device, dtype=torch.float32, non_blocking=True)

                

                logits = model(images)
                loss_bce_t  = criterion(logits, gt_masks)
                loss_dice_t = soft_dice_loss(logits, gt_masks)
                loss_t = loss_bce_t * lambda_bce + lambda_dice * loss_dice_t

                IOU_loss = iou_loss(logits, gt_masks)
                precision, recall = precision_recall(logits, gt_masks)

                ## average loss
                batch_sz = images.size(0)
                test_total_loss_sum += loss_t.item() * batch_sz
                test_bce_loss_sum   += loss_bce_t.item() * batch_sz
                test_dice_loss_sum  += loss_dice_t.item() * batch_sz
                IOU_loss_sum += IOU_loss.item() * batch_sz
                precision_sum += precision * batch_sz
                recall_sum += recall * batch_sz

                n_val += batch_sz

                val_bar.set_postfix({'Val Loss': test_total_loss_sum / max(1, n_val), 'Val BCE Loss': test_bce_loss_sum / max(1, n_val), 'Val Dice Loss': test_dice_loss_sum / max(1, n_val), 'Val dice coef': 1 - (test_dice_loss_sum / max(1, n_val)), 'Val IOU Loss': IOU_loss_sum / max(1, n_val), 'Val IOU Score': 1 - (IOU_loss_sum / max(1, n_val)), 'Precision': precision_sum / max(1, n_val), 'Recall': recall_sum / max(1, n_val)})
            end = time.time()
            test_epoch_time = end - start
            val_loss = test_total_loss_sum / max(1, n_val)
            val_dice_loss = test_dice_loss_sum / max(1, n_val)
            loss_bce_test = test_bce_loss_sum / max(1, n_val)
            val_coeff = 1 - val_dice_loss
            avg_IOU_loss = IOU_loss_sum / max(1, n_val)
            avg_IOU_score = 1 - avg_IOU_loss
            avg_precision = precision_sum / max(1, n_val)
            avg_recall = recall_sum / max(1, n_val)

            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_coeff)
            else:
                scheduler.step()

            if val_coeff > coef_to_beat:
                coef_to_beat = val_coeff
                model_to_path = f'{path}/{run_type}_final.pth'
                torch.save(model.state_dict(), model_to_path)
                
            
                
        

            ### logging ###
            testinglogs["time"].append(test_epoch_time)
            testinglogs["epoch"].append(epoch)
            testinglogs["total_loss"].append(val_loss)
            testinglogs["activation_func_loss"].append(loss_bce_test)
            testinglogs["dice_loss"].append(val_dice_loss)
            testinglogs["dice_coef"].append(1 - val_dice_loss)
            testinglogs["Iou_loss"].append(avg_IOU_loss)
            testinglogs["Iou_score"].append(avg_IOU_score)
            testinglogs["precision"].append(avg_precision)
            testinglogs["recall"].append(avg_recall)


            testinglogs["lr"].append(lr)

            early_stopper(val_loss, model)
            if early_stopper.early_stop:
                print("Early stopping triggered.")
                
                break

            print(f'Val loss: {val_loss:.4f}, BCE Loss: {loss_bce_test:.4f}, Dice Loss: {val_dice_loss:.4f}, Dice Coef: {1 - val_dice_loss:.4f}, IOU Loss: {avg_IOU_loss:.4f}, IOU Score: {avg_IOU_score:.4f}, Precision: {avg_precision:.4f}, Recall: {avg_recall:.4f}')
            print(f'Learning Rate: {lr:.6f}')

    print('Finished Training')
    return traininglogs, testinglogs

## Resolution experiment loop

In [32]:

resolutions_list = [64, 128, 256, 320, 512]

In [33]:
validation_res_list = [128, 512, 768]

In [34]:
def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()
def resolutionconfig(config,resolution,valres):
    # Models
    # set seed
    torch.manual_seed(config["random_seed"])
    np.random.seed(config["random_seed"])
    random.seed(config["random_seed"])
    g = torch.Generator(device='cpu')  # MPS backend doesn't use CUDA generators
    g.manual_seed(config["random_seed"])

    

    res_model = create_model("deeplabv3plus", "resnet34")

    res_optimizer = optim.Adam(
        res_model.parameters(),
        lr=config["learning_rate"],
        #momentum=config["momentum"]
    )

    res_criterion = nn.BCEWithLogitsLoss()
   # transformers
    if resolution == 64:
        transformer = albumentations.Compose([albumentations.Resize(64, 64),
                                             albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                             ToTensorV2()])
    elif resolution == 128:
        transformer = albumentations.Compose([albumentations.Resize(128, 128),
                                             albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                             ToTensorV2()])
    elif resolution == 256:
        transformer = albumentations.Compose([albumentations.Resize(256, 256),
                                             albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                             ToTensorV2()])
    elif resolution == 320:
        transformer = albumentations.Compose([albumentations.Resize(320, 320),
                                             albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                             ToTensorV2()])
    elif resolution == 512:
        transformer = albumentations.Compose([albumentations.Resize(512, 512),
                                             albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                             ToTensorV2()])

    val_transformer = albumentations.Compose([albumentations.Resize(valres, valres),
                                             albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                             ToTensorV2()])
        
    
    train_ds = Kvasir_Dataset(
        train_imgs,
        train_masks,
        images_directory,
        masks_directory,
        transformer)




    val_ds = Kvasir_Dataset(
        val_imgs,
        val_masks,
        images_directory,
        masks_directory,
        val_transformer)

    trainload, testload = loaders(config, train_ds, val_ds)


    return res_model, trainload, testload, res_optimizer, res_criterion

In [35]:
def res_exp(res, valres):
    path = f'Resolution_logs/{valres}'
    
    model_name = f"deeplabv3plus_res_{res}val"
    model_r, trainload, testload, optimizer_r, criterion_r = resolutionconfig(config,res, valres)
    training_logs, testing_logs = train(trainload, testload, model_r, criterion_r, optimizer_r, device, config, path, model_name)

    traininglogs_df = pd.DataFrame(training_logs)
    testinglogs_df  = pd.DataFrame(testing_logs)

    traininglogs_df["epoch"] = pd.to_numeric(traininglogs_df["epoch"], errors="raise").astype(int)
    testinglogs_df["epoch"]  = pd.to_numeric(testinglogs_df["epoch"],  errors="raise").astype(int)

    traininglogs_df.drop_duplicates(subset=["epoch"], keep="first", inplace=True)
    testinglogs_df.drop_duplicates(subset=["epoch"],  keep="first", inplace=True)

    experiment_logs = pd.merge(
        traininglogs_df, 
        testinglogs_df, 
        on="epoch", 
        how="inner", 
        suffixes=('_train', '_test')
    ).reset_index(drop=True)

    experiment_logs.to_csv(f'Resolution_logs/{valres}/{model_name}_logs.csv', index=False)

In [ ]:
for valres in validation_res_list:
    for res in resolutions_list:
        res_exp(res, valres)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Train 1/100:   0%|          | 0/88 [00:00<?, ?it/s]

Train Epoch [1/100] - Loss: 0.5320, BCE Loss: 0.3592, Dice Loss: 0.5547, Dice Coef: 0.4453, IOU Loss: 0.6054, IOU Score: 0.3946, Precision: 0.5310, Recall: 0.6371


Val   1/100:   0%|          | 0/120 [00:00<?, ?it/s]

Val loss: 0.7279, BCE Loss: 0.5660, Dice Loss: 0.7165, Dice Coef: 0.2835, IOU Loss: 0.8149, IOU Score: 0.1851, Precision: 0.5604, Recall: 0.2370
Learning Rate: 0.001000


...

Train 7/100:   0%|          | 0/88 [00:00<?, ?it/s]

Train Epoch [7/100] - Loss: 0.2277, BCE Loss: 0.1689, Dice Loss: 0.2287, Dice Coef: 0.7713, IOU Loss: 0.3005, IOU Score: 0.6995, Precision: 0.8400, Recall: 0.8026


Val   7/100:   0%|          | 0/120 [00:00<?, ?it/s]

Val loss: 0.6055, BCE Loss: 0.5631, Dice Loss: 0.5432, Dice Coef: 0.4568, IOU Loss: 0.6430, IOU Score: 0.3570, Precision: 0.7351, Recall: 0.4253
Learning Rate: 0.001000


Train 8/100:   0%|          | 0/88 [00:00<?, ?it/s]

# Finding res sweet spot

In [45]:
target_resolution_range = [288, 320, 352, 384, 416]

In [46]:

def restargetconfig(config,resolution,valres):
    # Models
    # set seed
    torch.manual_seed(config["random_seed"])
    np.random.seed(config["random_seed"])
    random.seed(config["random_seed"])
    g = torch.Generator(device='cpu')  # MPS backend doesn't use CUDA generators
    g.manual_seed(config["random_seed"])

    

    res_model = create_model("deeplabv3plus", "resnet34")

    res_optimizer = optim.Adam(
        res_model.parameters(),
        lr=config["learning_rate"],
        #momentum=config["momentum"]
    )

    res_criterion = nn.BCEWithLogitsLoss()
   # transformers
    
    transformer = albumentations.Compose([albumentations.Resize(resolution, resolution),
                                            albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                            ToTensorV2()])


    val_transformer = albumentations.Compose([albumentations.Resize(valres, valres),
                                             albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                             ToTensorV2()])
        
    
    train_ds = Kvasir_Dataset(
        train_imgs,
        train_masks,
        images_directory,
        masks_directory,
        transformer)




    val_ds = Kvasir_Dataset(
        val_imgs,
        val_masks,
        images_directory,
        masks_directory,
        val_transformer)

    trainload, testload = loaders(config, train_ds, val_ds)


    return res_model, trainload, testload, res_optimizer, res_criterion

In [47]:
def res_exp(res, valres):
    path = f'Resolution_logs/resolution_tuning/{valres}'
    
    model_name = f"deeplabv3plus_res_{res}val"
    model_r, trainload, testload, optimizer_r, criterion_r = restargetconfig(config,res, valres)
    training_logs, testing_logs = train(trainload, testload, model_r, criterion_r, optimizer_r, device, config, path, model_name)

    traininglogs_df = pd.DataFrame(training_logs)
    testinglogs_df = pd.DataFrame(testing_logs)
    experiment_logs = pd.merge(traininglogs_df, testinglogs_df, on="epoch", suffixes=('_train', '_test')).to_dict(orient='list')
    experiment_logs = pd.DataFrame(experiment_logs)
    experiment_logs.reset_index(drop=True, inplace=True)
    # export logs to csv to runs folder
    experiment_logs.to_csv(f'Resolution_logs/resolution_tuning/{valres}/{model_name}_logs.csv', index=False)

In [48]:
for valtarget in target_resolution_range:
    for samere_res in target_resolution_range:
        res_exp(samere_res, valtarget)

Epochs:   0%|          | 0/40 [00:00<?, ?it/s]

Train 1/40:   0%|          | 0/88 [00:00<?, ?it/s]

Train Epoch [1/40] - Loss: 0.3932, BCE Loss: 0.2799, Dice Loss: 0.4018, Dice Coef: 0.5982, IOU Loss: 0.4374, IOU Score: 0.5626, Precision: 0.7009, Recall: 0.7341


Val   1/40:   0%|          | 0/120 [00:00<?, ?it/s]

Val loss: 0.3818, BCE Loss: 0.3151, Dice Loss: 0.3654, Dice Coef: 0.6346, IOU Loss: 0.4438, IOU Score: 0.5562, Precision: 0.7933, Recall: 0.6658
Learning Rate: 0.001000


...

Val loss: 0.2356, BCE Loss: 0.2879, Dice Loss: 0.1721, Dice Coef: 0.8279, IOU Loss: 0.2469, IOU Score: 0.7531, Precision: 0.8751, Recall: 0.8480
Learning Rate: 0.000004


Train 40/40:   0%|          | 0/88 [00:00<?, ?it/s]

Train Epoch [40/40] - Loss: 0.0298, BCE Loss: 0.0196, Dice Loss: 0.0314, Dice Coef: 0.9686, IOU Loss: 0.0514, IOU Score: 0.9486, Precision: 0.9782, Recall: 0.9786


Val   40/40:   0%|          | 0/120 [00:00<?, ?it/s]

Val loss: 0.2306, BCE Loss: 0.2787, Dice Loss: 0.1701, Dice Coef: 0.8299, IOU Loss: 0.2453, IOU Score: 0.7547, Precision: 0.8724, Recall: 0.8520
Learning Rate: 0.000004
Finished Training


# Image predictions

In [ ]:
##rand
# Use the trained model for predictions (no file load)
prediction_model = model.to(config["device"])
prediction_model.eval()
print("Using trained model from this session.")

In [ ]:
# Show sample predictions with overlays
sample_indices = [3, 8, 13, 18, 23]

fig, axes = plt.subplots(len(sample_indices), 4, figsize=(20, 20))

for i, idx in enumerate(sample_indices):
    # Load image and mask
    image = cv2.imread(os.path.join(images_directory, val_imgs[idx]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    mask = cv2.imread(os.path.join(masks_directory, val_masks[idx]), cv2.IMREAD_GRAYSCALE)
    mask = mask / 255.0
    
    # Get prediction
    img_tensor = val_transform(image=image)["image"].unsqueeze(0).to(config["device"])
    with torch.no_grad():
        pred_logits = prediction_model(img_tensor)
        pred_mask = torch.sigmoid(pred_logits)[0, 0].cpu().numpy()
    
    # Resize prediction to match original image size
    pred_mask_resized = cv2.resize(pred_mask, (image.shape[1], image.shape[0]))
    
    # Create overlays
    overlay_gt = image.copy()
    overlay_gt[mask > 0.5] = [255, 0, 0]  # Red for ground truth
    
    overlay_pred = image.copy()
    overlay_pred[pred_mask_resized > 0.5] = [0, 255, 0]  # Green for prediction
    
    # Plot
    axes[i, 0].imshow(image)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis("off")
    
    axes[i, 1].imshow(overlay_gt)
    axes[i, 1].set_title("Ground Truth (Red)")
    axes[i, 1].axis("off")
    
    axes[i, 2].imshow(overlay_pred)
    axes[i, 2].set_title("Prediction (Green)")
    axes[i, 2].axis("off")
    
    axes[i, 3].imshow(pred_mask_resized, cmap="gray")
    axes[i, 3].set_title("Prediction Mask")
    axes[i, 3].axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# Calculate Dice Score for validation images
dice_scores = []

for idx in range(len(val_imgs)):
    # Load image and mask
    image = cv2.imread(os.path.join(images_directory, val_imgs[idx]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    mask = cv2.imread(os.path.join(masks_directory, val_masks[idx]), cv2.IMREAD_GRAYSCALE)
    mask = mask / 255.0
    
    # Get prediction
    img_tensor = val_transform(image=image)["image"].unsqueeze(0).to(config["device"])
    with torch.no_grad():
        pred_logits = prediction_model(img_tensor)
        pred_mask = torch.sigmoid(pred_logits)[0, 0].cpu().numpy()
    
    # Resize prediction to match original mask size
    pred_mask_resized = cv2.resize(pred_mask, (mask.shape[1], mask.shape[0]))
    
    # Calculate Dice Score
    mask_binary = mask > 0.5
    pred_binary = pred_mask_resized > 0.5
    
    intersection = np.logical_and(mask_binary, pred_binary).sum()
    dice = (2.0 * intersection) / (mask_binary.sum() + pred_binary.sum()) if (mask_binary.sum() + pred_binary.sum()) > 0 else 0
    dice_scores.append(dice)

print(f"Mean Dice Score: {np.mean(dice_scores):.4f}")
print(f"Std Dice Score: {np.std(dice_scores):.4f}")
print(f"Min Dice Score: {np.min(dice_scores):.4f}")
print(f"Max Dice Score: {np.max(dice_scores):.4f}")


In [ ]:
# Plot Dice Score distribution
plt.figure(figsize=(10, 6))
plt.hist(dice_scores, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Dice Score')
plt.ylabel('Number of Images')
plt.title('Distribution of Dice Scores on Validation Set')
plt.axvline(np.mean(dice_scores), color='red', linestyle='--', label=f'Mean: {np.mean(dice_scores):.3f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Show best and worst predictions
best_idx = np.argmax(dice_scores)
worst_idx = np.argmin(dice_scores)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for row, idx in enumerate([best_idx, worst_idx]):
    # Load image and mask
    image = cv2.imread(os.path.join(images_directory, val_imgs[idx]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    mask = cv2.imread(os.path.join(masks_directory, val_masks[idx]), cv2.IMREAD_GRAYSCALE)
    mask = mask / 255.0
    
    # Get prediction
    img_tensor = val_transform(image=image)["image"].unsqueeze(0).to(config["device"])
    with torch.no_grad():
        pred_logits = prediction_model(img_tensor)
        pred_mask = torch.sigmoid(pred_logits)[0, 0].cpu().numpy()
    
    # Resize prediction to match original image size
    pred_mask_resized = cv2.resize(pred_mask, (image.shape[1], image.shape[0]))
    
    # Plot
    axes[row, 0].imshow(image)
    axes[row, 0].set_title("Original Image")
    axes[row, 0].axis("off")
    
    axes[row, 1].imshow(mask, cmap="gray")
    axes[row, 1].set_title("Ground Truth")
    axes[row, 1].axis("off")
    
    axes[row, 2].imshow(pred_mask_resized, cmap="gray")
    axes[row, 2].set_title(f"Prediction (Dice: {dice_scores[idx]:.3f})")
    axes[row, 2].axis("off")

axes[0, 0].set_ylabel("Best Prediction", fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel("Worst Prediction", fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
# Calculate accuracy, precision, recall
all_accuracies = []
all_precisions = []
all_recalls = []

for idx in range(len(val_imgs)):
    mask = cv2.imread(os.path.join(masks_directory, val_masks[idx]), cv2.IMREAD_GRAYSCALE)
    mask = mask / 255.0
    
    image = cv2.imread(os.path.join(images_directory, val_imgs[idx]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    img_tensor = val_transform(image=image)["image"].unsqueeze(0).to(config["device"])
    with torch.no_grad():
        pred_logits = prediction_model(img_tensor)
        pred_mask = torch.sigmoid(pred_logits)[0, 0].cpu().numpy()
    
    # Resize prediction to match original mask size
    pred_mask_resized = cv2.resize(pred_mask, (mask.shape[1], mask.shape[0]))
    
    # Binary masks
    mask_binary = mask > 0.5
    pred_binary = pred_mask_resized > 0.5
    
    # Calculate metrics
    tp = np.logical_and(mask_binary, pred_binary).sum()
    fp = np.logical_and(~mask_binary, pred_binary).sum()
    fn = np.logical_and(mask_binary, ~pred_binary).sum()
    tn = np.logical_and(~mask_binary, ~pred_binary).sum()
    
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    all_accuracies.append(accuracy)
    all_precisions.append(precision)
    all_recalls.append(recall)

print("Overall Metrics:")
print(f"Accuracy: {np.mean(all_accuracies):.4f}")
print(f"Precision: {np.mean(all_precisions):.4f}")
print(f"Recall: {np.mean(all_recalls):.4f}")


In [ ]:
# Plot all metrics together
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics_data = {
    'Accuracy': all_accuracies,
    'Precision': all_precisions,
    'Recall': all_recalls
}

for idx, (metric_name, values) in enumerate(metrics_data.items()):
    axes[idx].hist(values, bins=20, edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(metric_name)
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'{metric_name} Distribution')
    axes[idx].axvline(np.mean(values), color='red', linestyle='--', 
                     label=f'Mean: {np.mean(values):.3f}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Error analysis - show where model makes mistakes
medium_idx = np.argsort(dice_scores)[len(dice_scores)//2]  # middle performing image

image = cv2.imread(os.path.join(images_directory, val_imgs[medium_idx]))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

mask = cv2.imread(os.path.join(masks_directory, val_masks[medium_idx]), cv2.IMREAD_GRAYSCALE)
mask = mask / 255.0

img_tensor = val_transform(image=image)["image"].unsqueeze(0).to(config["device"])
with torch.no_grad():
    pred_logits = prediction_model(img_tensor)
    pred_mask = torch.sigmoid(pred_logits)[0, 0].cpu().numpy()

# Resize prediction to match original image size
pred_mask_resized = cv2.resize(pred_mask, (image.shape[1], image.shape[0]))

# Create error visualization
pred_binary = pred_mask_resized > 0.5
true_binary = mask > 0.5

# False positives (model predicts but shouldn't)
fp_mask = np.logical_and(pred_binary, ~true_binary)
# False negatives (model misses)
fn_mask = np.logical_and(~pred_binary, true_binary)

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(image)
axes[0].set_title("Original Image")
axes[0].axis("off")

axes[1].imshow(true_binary, cmap="gray")
axes[1].set_title("Ground Truth")
axes[1].axis("off")

axes[2].imshow(pred_binary, cmap="gray")
axes[2].set_title("Prediction")
axes[2].axis("off")

# Show errors overlayed on image
error_overlay = image.copy()
error_overlay[fp_mask] = [255, 0, 0]  # Red for false positives
error_overlay[fn_mask] = [0, 0, 255]  # Blue for false negatives
axes[3].imshow(error_overlay)
axes[3].set_title("Errors (Red=FP, Blue=FN)")
axes[3].axis("off")

plt.tight_layout()
plt.show()


# Exploration of logs

## Validation stages

In [63]:
directory = "Resolution_logs"

In [65]:
def createdf(valres):
    directory = "Resolution_logs"
    dir = os.path.join(directory+f'/{valres}','*.csv')
    all_logs = {}
    for filepath in glob.glob(dir):
        df = pd.read_csv(filepath)
        model_name = os.path.basename(filepath).replace('_logs.csv','')
        all_logs[model_name] = df
    return all_logs
    

    

In [66]:
logs_128 = createdf(128)
logs_512 = createdf(512)
logs_768 = createdf(768)

In [67]:
for table in logs_128.values():
    display(table.describe())

,time_train,lambda_bce_train,lambda_dice_train,epoch,total_loss_train,activation_func_loss_train,dice_loss_train,dice_coef_train,Iou_loss_train,Iou_score_train,...,lambda_dice_test,total_loss_test,activation_func_loss_test,dice_loss_test,dice_coef_test,Iou_loss_test,Iou_score_test,precision_test,recall_test,lr_test
count,40.000000,40.0,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,...,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,4.000000e+01
mean,10.522535,0.4,0.7,19.500000,0.084970,0.064740,0.084392,0.915608,0.120301,0.879699,...,0.7,0.608309,0.770259,0.428865,0.571135,0.504031,0.495969,0.554781,0.898970,2.999268e-04
std,0.074249,0.0,0.0,11.690452,0.074044,0.057846,0.072965,0.072965,0.081421,0.081421,...,0.0,0.068426,0.167592,0.024598,0.024598,0.026325,0.026325,0.033694,0.051384,3.890078e-04
min,10.416694,0.4,0.7,0.000000,0.043020,0.030164,0.044200,0.600280,0.071171,0.564611,...,0.7,0.425463,0.344985,0.386075,0.480855,0.450586,0.412541,0.459542,0.692847,9.765625e-07
25%,10.468105,0.4,0.7,9.750000,0.043757,0.030854,0.044843,0.915543,0.072287,0.873102,...,0.7,0.560347,0.654069,0.414361,0.565952,0.489851,0.487562,0.533902,0.893446,7.812500e-06
50%,10.509624,0.4,0.7,19.500000,0.048384,0.034831,0.049250,0.950750,0.078735,0.921265,...,0.7,0.628415,0.827911,0.423363,0.576637,0.500664,0.499336,0.548223,0.919524,6.250000e-05
75%,10.552230,0.4,0.7,29.250000,0.086072,0.067380,0.084457,0.955157,0.126898,0.927713,...,0.7,0.654006,0.890705,0.434048,0.585639,0.512438,0.510149,0.573561,0.926235,5.000000e-04
max,10.764962,0.4,0.7,39.000000,0.391605,0.279503,0.399720,0.955800,0.435389,0.928829,...,0.7,0.709509,0.997869,0.519145,0.613925,0.587459,0.549414,0.654530,0.932452,1.000000e-03


,time_train,lambda_bce_train,lambda_dice_train,epoch,total_loss_train,activation_func_loss_train,dice_loss_train,dice_coef_train,Iou_loss_train,Iou_score_train,...,lambda_dice_test,total_loss_test,activation_func_loss_test,dice_loss_test,dice_coef_test,Iou_loss_test,Iou_score_test,precision_test,recall_test,lr_test
count,40.000000,40.0,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,...,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,4.000000e+01
mean,31.994412,0.4,0.7,19.500000,0.080080,0.060688,0.079721,0.920279,0.113798,0.886202,...,0.7,1.041492,1.321286,0.732826,0.267174,0.821214,0.178786,0.432831,0.597092,2.249695e-04
std,0.226320,0.0,0.0,11.690452,0.077548,0.060414,0.076462,0.076462,0.086010,0.086010,...,0.0,0.137785,0.348741,0.025609,0.025609,0.024133,0.024133,0.108077,0.106976,3.571809e-04
min,31.792432,0.4,0.7,0.000000,0.040975,0.028269,0.042320,0.590311,0.068208,0.550019,...,0.7,0.644093,0.496469,0.636436,0.193648,0.705630,0.117308,0.226762,0.207806,2.441406e-07
25%,31.841567,0.4,0.7,9.750000,0.041765,0.029195,0.042988,0.931452,0.069159,0.894490,...,0.7,1.021146,1.239391,0.722221,0.258127,0.813289,0.170814,0.371365,0.560683,3.417969e-06
50%,31.921747,0.4,0.7,19.500000,0.044727,0.031953,0.045637,0.954363,0.073419,0.926581,...,0.7,1.072366,1.408346,0.731601,0.268399,0.822399,0.177601,0.423129,0.612755,3.125000e-05
75%,32.075792,0.4,0.7,29.250000,0.070038,0.055134,0.068548,0.957012,0.105510,0.930841,...,0.7,1.101639,1.480735,0.741873,0.277779,0.829186,0.186711,0.465890,0.651332,2.500000e-04
max,32.879746,0.4,0.7,39.000000,0.402253,0.288678,0.409689,0.957680,0.449981,0.931792,...,0.7,1.556447,2.632760,0.806352,0.363564,0.882692,0.294370,0.805222,0.856093,1.000000e-03


,time_train,lambda_bce_train,lambda_dice_train,epoch,total_loss_train,activation_func_loss_train,dice_loss_train,dice_coef_train,Iou_loss_train,Iou_score_train,...,lambda_dice_test,total_loss_test,activation_func_loss_test,dice_loss_test,dice_coef_test,Iou_loss_test,Iou_score_test,precision_test,recall_test,lr_test
count,40.000000,40.0,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,...,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,4.467267,0.4,0.7,19.500000,0.129989,0.090291,0.134104,0.865896,0.182342,0.817658,...,0.7,0.579030,0.594922,0.487230,0.512770,0.604208,0.395792,0.584138,0.624513,0.000500
std,0.075409,0.0,0.0,11.690452,0.107387,0.077610,0.109192,0.109192,0.122437,0.122437,...,0.0,0.040080,0.098952,0.045753,0.045753,0.039327,0.039327,0.068455,0.094435,0.000439
min,4.378865,0.4,0.7,0.000000,0.054044,0.033545,0.058037,0.453377,0.087768,0.404135,...,0.7,0.505710,0.397421,0.443339,0.351026,0.550973,0.251968,0.451227,0.386917,0.000004
25%,4.422306,0.4,0.7,9.750000,0.056653,0.035607,0.060600,0.818959,0.091992,0.749809,...,0.7,0.555596,0.501756,0.460693,0.513190,0.582801,0.390572,0.542299,0.573237,0.000055
50%,4.451849,0.4,0.7,19.500000,0.075720,0.049771,0.079730,0.920270,0.122145,0.877855,...,0.7,0.583489,0.610581,0.472927,0.527073,0.598770,0.401230,0.564269,0.663633,0.000500
75%,4.480419,0.4,0.7,29.250000,0.178472,0.128264,0.181041,0.939400,0.250191,0.908008,...,0.7,0.613847,0.687273,0.486810,0.539307,0.609428,0.417199,0.616657,0.676337,0.001000
max,4.798667,0.4,0.7,39.000000,0.524496,0.354651,0.546623,0.941963,0.595865,0.912232,...,0.7,0.679309,0.723093,0.648974,0.556661,0.748032,0.449027,0.743542,0.786017,0.001000


,time_train,lambda_bce_train,lambda_dice_train,epoch,total_loss_train,activation_func_loss_train,dice_loss_train,dice_coef_train,Iou_loss_train,Iou_score_train,...,lambda_dice_test,total_loss_test,activation_func_loss_test,dice_loss_test,dice_coef_test,Iou_loss_test,Iou_score_test,precision_test,recall_test,lr_test
count,40.000000,40.0,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,...,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,4.000000e+01
mean,15.788081,0.4,0.7,19.500000,0.083334,0.063055,0.083017,0.916983,0.117794,0.882206,...,0.7,0.717209,0.860653,0.532782,0.467218,0.605124,0.394876,0.462420,0.878972,2.999390e-04
std,0.374599,0.0,0.0,11.690452,0.075484,0.059876,0.073850,0.073850,0.083438,0.083438,...,0.0,0.114133,0.246672,0.057921,0.057921,0.057912,0.057912,0.117139,0.154342,4.051421e-04
min,15.296376,0.4,0.7,0.000000,0.041231,0.027834,0.042992,0.610732,0.068726,0.573553,...,0.7,0.456819,0.299263,0.472313,0.128797,0.515963,0.086171,0.377046,0.169007,4.882813e-07
25%,15.390580,0.4,0.7,9.750000,0.041907,0.028377,0.043654,0.920240,0.069641,0.880243,...,0.7,0.661959,0.751087,0.518129,0.462232,0.587149,0.381025,0.393994,0.878030,6.835938e-06
50%,15.869582,0.4,0.7,19.500000,0.045951,0.031877,0.047429,0.952571,0.075208,0.924792,...,0.7,0.752990,0.956326,0.529620,0.470380,0.607164,0.392836,0.408932,0.947055,6.250000e-05
75%,16.082885,0.4,0.7,29.250000,0.081045,0.063034,0.079760,0.956346,0.119757,0.930359,...,0.7,0.788217,1.031701,0.537768,0.481871,0.618975,0.412851,0.459808,0.953118,5.000000e-04
max,16.671862,0.4,0.7,39.000000,0.382368,0.274701,0.389268,0.957008,0.426447,0.931274,...,0.7,0.921916,1.145936,0.871203,0.527687,0.913829,0.484037,0.894245,0.961260,1.000000e-03


,time_train,lambda_bce_train,lambda_dice_train,epoch,total_loss_train,activation_func_loss_train,dice_loss_train,dice_coef_train,Iou_loss_train,Iou_score_train,...,lambda_dice_test,total_loss_test,activation_func_loss_test,dice_loss_test,dice_coef_test,Iou_loss_test,Iou_score_test,precision_test,recall_test,lr_test
count,40.000000,40.0,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,...,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,5.649700,0.4,0.7,19.500000,0.099045,0.071470,0.100653,0.899347,0.140371,0.859629,...,0.7,0.305339,0.330598,0.247285,0.752715,0.329995,0.670005,0.790911,0.829977,0.000419
std,0.064506,0.0,0.0,11.690452,0.086588,0.066399,0.085951,0.085951,0.097095,0.097095,...,0.0,0.038741,0.033669,0.052284,0.052284,0.052380,0.052380,0.039079,0.051152,0.000433
min,5.499574,0.4,0.7,0.000000,0.042871,0.026639,0.046021,0.537908,0.072085,0.493022,...,0.7,0.267852,0.251029,0.214245,0.505002,0.294128,0.414305,0.654427,0.565909,0.000002
25%,5.617567,0.4,0.7,9.750000,0.044444,0.027819,0.047601,0.870475,0.074844,0.814817,...,0.7,0.285991,0.303747,0.222392,0.745895,0.306356,0.666308,0.793365,0.834645,0.000027
50%,5.644085,0.4,0.7,19.500000,0.055766,0.036766,0.058657,0.941343,0.091647,0.908353,...,0.7,0.300338,0.339677,0.225499,0.774501,0.310481,0.689519,0.801206,0.844432,0.000250
75%,5.695911,0.4,0.7,29.250000,0.132349,0.101527,0.129525,0.952399,0.185183,0.925156,...,0.7,0.305295,0.359686,0.254105,0.777608,0.333692,0.693644,0.809605,0.850078,0.001000
max,5.804618,0.4,0.7,39.000000,0.450664,0.318000,0.462092,0.953979,0.506978,0.927915,...,0.7,0.502487,0.389972,0.494998,0.785755,0.585695,0.705872,0.833899,0.880730,0.001000


In [68]:
for table in logs_128:
    print(str(table))

deeplabv3plus_res_256val
deeplabv3plus_res_512val
deeplabv3plus_res_64val
deeplabv3plus_res_320val
deeplabv3plus_res_128val


In [165]:
def returnstats(listoftables,valres):
    names = []
    for table in listoftables:
        names.append(str(table))
    summaries = {f"val_on_{valres}":[],"tain_Time_avg":[],"dice_coef_test_max": [],"dice_coef_test_min":[],"dice_coef_test_mean": [], "Iou_score_test_max": [], "Iou_score_test_min": [], "Iou_score_test_mean": [], "precision_test_max": [], "precision_test_min": [], "precision_test_mean": [], "recall_test_max": [], "recall_test_min": [], "recall_test_mean": []}
   
    cname = 0
    for table in listoftables.values():
        
        name = names[cname]
        name = name.replace('val','on_train')
        summaries[f"val_on_{valres}"].append(name)
        summaries["dice_coef_test_max"].append(table["dice_coef_test"].max())
        summaries["Iou_score_test_max"].append(table["Iou_score_test"].max())
        summaries["precision_test_max"].append(table["precision_test"].max())
        summaries["recall_test_max"].append(table["recall_test"].max())

        summaries["dice_coef_test_min"].append(table["dice_coef_test"].min())
        summaries["Iou_score_test_min"].append(table["Iou_score_test"].min())
        summaries["precision_test_min"].append(table["precision_test"].min())
        summaries["recall_test_min"].append(table["recall_test"].min())

        summaries["tain_Time_avg"].append(table["time_train"].mean())
        summaries["dice_coef_test_mean"].append(table["dice_coef_test"].mean())
        summaries["Iou_score_test_mean"].append(table["Iou_score_test"].mean())
        summaries["precision_test_mean"].append(table["precision_test"].mean())
        summaries["recall_test_mean"].append(table["recall_test"].mean())
        cname +=1

    return pd.DataFrame(summaries) 
        
        

In [166]:
val_128 = returnstats(logs_128,128)
val_512 = returnstats(logs_512,512)
val_768 = returnstats(logs_768,768)

In [167]:
val_128

,val_on_128,tain_Time_avg,dice_coef_test_max,dice_coef_test_min,dice_coef_test_mean,Iou_score_test_max,Iou_score_test_min,Iou_score_test_mean,precision_test_max,precision_test_min,precision_test_mean,recall_test_max,recall_test_min,recall_test_mean
0,deeplabv3plus_res_256on_train,10.522535,0.613925,0.480855,0.571135,0.549414,0.412541,0.495969,0.654530,0.459542,0.554781,0.932452,0.692847,0.898970
1,deeplabv3plus_res_512on_train,31.994412,0.363564,0.193648,0.267174,0.294370,0.117308,0.178786,0.805222,0.226762,0.432831,0.856093,0.207806,0.597092
2,deeplabv3plus_res_64on_train,4.467267,0.556661,0.351026,0.512770,0.449027,0.251968,0.395792,0.743542,0.451227,0.584138,0.786017,0.386917,0.624513
3,deeplabv3plus_res_320on_train,15.788081,0.527687,0.128797,0.467218,0.484037,0.086171,0.394876,0.894245,0.377046,0.462420,0.961260,0.169007,0.878972
4,deeplabv3plus_res_128on_train,5.649700,0.785755,0.505002,0.752715,0.705872,0.414305,0.670005,0.833899,0.654427,0.790911,0.880730,0.565909,0.829977


In [168]:
val_512

,val_on_512,tain_Time_avg,dice_coef_test_max,dice_coef_test_min,dice_coef_test_mean,Iou_score_test_max,Iou_score_test_min,Iou_score_test_mean,precision_test_max,precision_test_min,precision_test_mean,recall_test_max,recall_test_min,recall_test_mean
0,deeplabv3plus_res_256on_train,10.871408,0.680407,0.461904,0.633761,0.579228,0.362269,0.534476,0.823881,0.655535,0.722004,0.811874,0.427431,0.704992
1,deeplabv3plus_res_512on_train,32.337263,0.848081,0.633405,0.816425,0.776312,0.556824,0.741870,0.892171,0.716416,0.854981,0.895648,0.736480,0.862227
2,deeplabv3plus_res_64on_train,4.518173,0.212155,0.038995,0.104586,0.115574,0.014006,0.054350,0.301004,0.142242,0.180943,0.206480,0.016081,0.094081
3,deeplabv3plus_res_320on_train,16.084421,0.800944,0.576300,0.756977,0.713455,0.487928,0.670312,0.869752,0.701016,0.814856,0.870985,0.611258,0.807912
4,deeplabv3plus_res_128on_train,6.157238,0.321029,0.171844,0.296092,0.218762,0.099930,0.193747,0.564964,0.255966,0.341323,0.571472,0.141120,0.425776


In [169]:
val_768

,val_on_768,tain_Time_avg,dice_coef_test_max,dice_coef_test_min,dice_coef_test_mean,Iou_score_test_max,Iou_score_test_min,Iou_score_test_mean,precision_test_max,precision_test_min,precision_test_mean,recall_test_max,recall_test_min,recall_test_mean
0,deeplabv3plus_res_256on_train,10.649744,0.496626,0.168452,0.427811,0.382995,0.101446,0.322245,0.673709,0.358214,0.512719,0.704248,0.143692,0.538774
1,deeplabv3plus_res_512on_train,32.814170,0.807427,0.536510,0.747768,0.720174,0.456857,0.659669,0.887257,0.683638,0.812428,0.859698,0.564810,0.794246
2,deeplabv3plus_res_64on_train,4.928469,0.138316,0.016195,0.049603,0.070375,0.005301,0.023328,0.209133,0.120441,0.144508,0.120141,0.005619,0.031851
3,deeplabv3plus_res_320on_train,15.982260,0.590481,0.400889,0.556680,0.480798,0.299798,0.448158,0.766844,0.597603,0.658655,0.734309,0.380417,0.642408
4,deeplabv3plus_res_128on_train,6.133313,0.207621,0.058091,0.142062,0.122354,0.014343,0.079856,0.348784,0.219071,0.246238,0.253802,0.015441,0.150552


## Finding Sweetspot for resolution

In [171]:
def createdf_1(valres):
    directory = "Resolution_logs/resolution_tuning"
    dir = os.path.join(directory+f'/{valres}','*.csv')
    all_logs = {}
    for filepath in glob.glob(dir):
        df = pd.read_csv(filepath)
        model_name = os.path.basename(filepath).replace('_logs.csv','')
        all_logs[model_name] = df
    return all_logs

In [172]:
logs_288_rt = createdf_1(288)
logs_320_rt = createdf_1(320)
logs_352_rt = createdf_1(352)
logs_384_rt = createdf_1(384)
logs_416_rt = createdf_1(416)

In [173]:
summaries_log_rt_288 = returnstats(logs_288_rt,288)
summaries_log_rt_320 = returnstats(logs_320_rt,320)
summaries_log_rt_352 = returnstats(logs_352_rt,352)
summaries_log_rt_384 = returnstats(logs_384_rt,384)
summaries_log_rt_416 = returnstats(logs_416_rt,416)

In [174]:
summaries_log_rt_288

,val_on_288,tain_Time_avg,dice_coef_test_max,dice_coef_test_min,dice_coef_test_mean,Iou_score_test_max,Iou_score_test_min,Iou_score_test_mean,precision_test_max,precision_test_min,precision_test_mean,recall_test_max,recall_test_min,recall_test_mean
0,deeplabv3plus_res_416on_train,24.091652,0.794880,0.589882,0.754010,0.724653,0.538043,0.685478,0.852780,0.692656,0.776196,0.916202,0.736868,0.857523
1,deeplabv3plus_res_384on_train,19.669948,0.816931,0.644377,0.778852,0.744316,0.563906,0.708542,0.876184,0.649182,0.793305,0.925709,0.701757,0.870551
2,deeplabv3plus_res_352on_train,18.425918,0.826799,0.556499,0.790866,0.752572,0.481535,0.713003,0.899824,0.658602,0.802109,0.924700,0.663320,0.880478
3,deeplabv3plus_res_288on_train,13.922148,0.839977,0.634636,0.810528,0.770483,0.556228,0.737266,0.879230,0.726479,0.845925,0.889381,0.665772,0.858145
4,deeplabv3plus_res_320on_train,15.837901,0.848612,0.599419,0.808556,0.778964,0.521720,0.735508,0.876874,0.623667,0.831773,0.914420,0.730676,0.873828


In [175]:
summaries_log_rt_320

,val_on_320,tain_Time_avg,dice_coef_test_max,dice_coef_test_min,dice_coef_test_mean,Iou_score_test_max,Iou_score_test_min,Iou_score_test_mean,precision_test_max,precision_test_min,precision_test_mean,recall_test_max,recall_test_min,recall_test_mean
0,deeplabv3plus_res_416on_train,24.073911,0.820836,0.628107,0.783995,0.746607,0.546963,0.709802,0.889347,0.737435,0.845291,0.869651,0.661502,0.828390
1,deeplabv3plus_res_384on_train,19.601001,0.832313,0.630679,0.795579,0.764202,0.544962,0.722038,0.910971,0.646131,0.817974,0.914905,0.731569,0.873018
2,deeplabv3plus_res_352on_train,18.297693,0.845983,0.654382,0.814334,0.776888,0.572994,0.742070,0.880528,0.746843,0.843032,0.913254,0.715942,0.873037
3,deeplabv3plus_res_288on_train,13.532039,0.841006,0.644525,0.807099,0.767643,0.551808,0.730719,0.880481,0.800227,0.853123,0.887491,0.634410,0.844889
4,deeplabv3plus_res_320on_train,15.743265,0.840587,0.631047,0.815918,0.768174,0.543995,0.741770,0.894886,0.708045,0.846515,0.893269,0.741356,0.868383


In [176]:
summaries_log_rt_352

,val_on_352,tain_Time_avg,dice_coef_test_max,dice_coef_test_min,dice_coef_test_mean,Iou_score_test_max,Iou_score_test_min,Iou_score_test_mean,precision_test_max,precision_test_min,precision_test_mean,recall_test_max,recall_test_min,recall_test_mean
0,deeplabv3plus_res_416on_train,24.030474,0.833769,0.647781,0.799399,0.763528,0.557976,0.728504,0.863383,0.673178,0.837172,0.893134,0.679150,0.852492
1,deeplabv3plus_res_384on_train,19.712489,0.848284,0.614144,0.816357,0.779779,0.517338,0.744718,0.875474,0.598947,0.840624,0.912011,0.759518,0.876542
2,deeplabv3plus_res_352on_train,18.219071,0.845476,0.640159,0.809760,0.773585,0.553673,0.737020,0.881849,0.704661,0.841973,0.904260,0.691572,0.862685
3,deeplabv3plus_res_288on_train,13.571012,0.832756,0.634491,0.796927,0.756562,0.547009,0.718656,0.877485,0.735108,0.837890,0.895327,0.714227,0.848213
4,deeplabv3plus_res_320on_train,15.679552,0.846881,0.643435,0.809398,0.775988,0.549876,0.734224,0.891497,0.774413,0.861883,0.885026,0.680963,0.843907


In [177]:
summaries_log_rt_384

,val_on_384,tain_Time_avg,dice_coef_test_max,dice_coef_test_min,dice_coef_test_mean,Iou_score_test_max,Iou_score_test_min,Iou_score_test_mean,precision_test_max,precision_test_min,precision_test_mean,recall_test_max,recall_test_min,recall_test_mean
0,deeplabv3plus_res_416on_train,24.035788,0.840387,0.617813,0.804952,0.767116,0.528739,0.730563,0.882630,0.628507,0.845806,0.912135,0.734951,0.852934
1,deeplabv3plus_res_384on_train,19.913538,0.842827,0.566397,0.811716,0.766669,0.473875,0.735810,0.888244,0.525930,0.835258,0.919107,0.714264,0.874405
2,deeplabv3plus_res_352on_train,18.230297,0.846333,0.628968,0.813987,0.775891,0.543188,0.740471,0.891952,0.730233,0.861957,0.893859,0.695509,0.847877
3,deeplabv3plus_res_288on_train,13.566100,0.816656,0.633649,0.784502,0.736743,0.541190,0.700612,0.898626,0.675125,0.847582,0.859977,0.665739,0.816493
4,deeplabv3plus_res_320on_train,15.761949,0.838471,0.647502,0.803028,0.762496,0.563542,0.723130,0.882347,0.742104,0.857496,0.875091,0.655626,0.837805


In [178]:
summaries_log_rt_416

,val_on_416,tain_Time_avg,dice_coef_test_max,dice_coef_test_min,dice_coef_test_mean,Iou_score_test_max,Iou_score_test_min,Iou_score_test_mean,precision_test_max,precision_test_min,precision_test_mean,recall_test_max,recall_test_min,recall_test_mean
0,deeplabv3plus_res_416on_train,24.028850,0.837437,0.638005,0.810550,0.762990,0.537924,0.735213,0.913938,0.652317,0.856369,0.882940,0.643300,0.844347
1,deeplabv3plus_res_384on_train,19.691946,0.851192,0.584564,0.818714,0.783564,0.515434,0.747684,0.891915,0.776985,0.860222,0.910159,0.616817,0.862419
2,deeplabv3plus_res_352on_train,18.313118,0.831147,0.529750,0.798121,0.756705,0.440911,0.720708,0.883893,0.726882,0.853098,0.881600,0.534248,0.831995
3,deeplabv3plus_res_288on_train,13.574577,0.825329,0.627501,0.777899,0.740935,0.539386,0.693390,0.876756,0.680359,0.834907,0.864384,0.692667,0.818265
4,deeplabv3plus_res_320on_train,15.708655,0.833792,0.616031,0.797736,0.754695,0.534052,0.716987,0.884852,0.729972,0.840831,0.912142,0.585371,0.843105


# Plots

### lines

In [156]:
def lines(logs):
    names = []
    for table in logs:
        name=str(table)
        name = name.replace('res_','trained_on_')
        name = name.replace('val','')
        name = name.replace('_',' ')
        names.append(name)
    counter=0
    for table in logs.values():
        fig = px.line(table, x='epoch', y=['dice_coef_test', 'Iou_score_test', 'precision_test', 'recall_test'],
              title=f'Model Performance over Epochs ({names[counter]})',
              labels={'value': 'Metric Value', 'epoch': 'Epoch'},
              template='plotly_dark')
        fig.update_layout(legend_title_text='Metrics')
        fig.show()
        counter+=1


In [157]:
lines(logs_128)

In [159]:
lines(logs_512)

In [160]:
lines(logs_768)

### Heatmaps

In [15]:
# ===============================
# Resolution Evaluation Utilities
# ===============================
import re
from collections import defaultdict
from datetime import datetime

EVAL_NORMALIZE_MEAN = (0.485, 0.456, 0.406)
EVAL_NORMALIZE_STD = (0.229, 0.224, 0.225)


def parse_resolution_from_name(checkpoint_path: Path) -> int:
    match = re.search(r"res_(\d+)", checkpoint_path.stem)
    if not match:
        raise ValueError(f"Unable to infer resolution from {checkpoint_path.name}")
    return int(match.group(1))


class KvasirResolutionEvalDataset(Dataset):
    def __init__(
        self,
        image_names: list[str],
        images_dir: Path,
        masks_dir: Path,
        resize_to: int,
    ) -> None:
        self.image_names = image_names
        self.images_dir = Path(images_dir)
        self.masks_dir = Path(masks_dir)
        self.resize_to = resize_to
        self.transform = albumentations.Compose(
            [
                albumentations.Resize(resize_to, resize_to),
                albumentations.Normalize(EVAL_NORMALIZE_MEAN, EVAL_NORMALIZE_STD),
                ToTensorV2(),
            ]
        )

    def __len__(self) -> int:
        return len(self.image_names)

    def __getitem__(self, idx: int):
        image_name = self.image_names[idx]
        image_path = self.images_dir / image_name
        mask_path = self.masks_dir / image_name

        image_bgr = cv2.imread(str(image_path), cv2.IMREAD_COLOR)
        if image_bgr is None:
            raise FileNotFoundError(f"Missing image: {image_path}")
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

        mask_raw = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if mask_raw is None:
            raise FileNotFoundError(f"Missing mask: {mask_path}")

        transformed = self.transform(image=image_rgb, mask=mask_raw)
        image_tensor = transformed["image"]
        mask_tensor = transformed["mask"]
        if mask_tensor.ndim == 2:
            mask_tensor = mask_tensor.unsqueeze(0)
        mask_tensor = mask_tensor.float()
        if mask_tensor.max() > 1:
            mask_tensor = (mask_tensor / 255.0).float()
        mask_tensor = (mask_tensor > 0.5).float()

        return image_tensor, mask_tensor, image_name


def eval_collate_fn(batch):
    images = torch.stack([item[0] for item in batch], dim=0)
    masks = torch.stack([item[1] for item in batch], dim=0)
    names = [item[2] for item in batch]
    return images, masks, names


def compute_binary_metrics(pred_mask: torch.Tensor, gt_mask: torch.Tensor, eps: float = 1e-7) -> dict[str, float]:
    preds = pred_mask.view(-1).float()
    targets = gt_mask.view(-1).float()
    intersection = torch.sum(preds * targets)
    pred_sum = torch.sum(preds)
    target_sum = torch.sum(targets)
    union = pred_sum + target_sum - intersection

    dice = (2 * intersection + eps) / (pred_sum + target_sum + eps)
    iou = (intersection + eps) / (union + eps)

    tp = intersection
    fp = torch.sum(preds * (1 - targets))
    fn = torch.sum((1 - preds) * targets)
    precision = (tp + eps) / (tp + fp + eps)
    recall = (tp + eps) / (tp + fn + eps)

    return {
        "dice": dice.item(),
        "iou": iou.item(),
        "precision": precision.item(),
        "recall": recall.item(),
    }



In [16]:
def load_model_from_checkpoint(checkpoint_path: Path, device: torch.device) -> nn.Module:
    checkpoint_path = Path(checkpoint_path).expanduser().resolve()
    model = create_model("deeplabv3plus", "resnet34")
    state_dict = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model


def build_resolution_loader(
    image_names: list[str],
    images_dir: Path,
    masks_dir: Path,
    resize_to: int,
    batch_size: int,
) -> DataLoader:
    dataset = KvasirResolutionEvalDataset(image_names, images_dir, masks_dir, resize_to)
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=config.get("workers", 0),
        collate_fn=eval_collate_fn,
    )
    return loader


def evaluate_checkpoint_simple(
    model: nn.Module,
    loader: DataLoader,
    resolution: int,
    checkpoint_path: Path,
    threshold: float,
    device: torch.device,
) -> tuple[list[dict[str, float]], list[dict]]:
    per_image_metrics: list[dict[str, float]] = []
    prediction_samples: list[dict] = []
    checkpoint_name = checkpoint_path.name

    model.eval()
    with torch.no_grad():
        for images, masks, names in loader:
            images = images.to(device=device, dtype=torch.float32, non_blocking=True)
            masks = masks.to(device=device, dtype=torch.float32, non_blocking=True)

            logits = model(images)
            probs = torch.sigmoid(logits)
            preds = (probs > threshold).float()

            preds_cpu = preds.detach().cpu()
            masks_cpu = masks.detach().cpu()

            batch_size = preds_cpu.size(0)
            for idx in range(batch_size):
                name = names[idx]
                metrics = compute_binary_metrics(preds_cpu[idx], masks_cpu[idx])
                record = {
                    "checkpoint": checkpoint_name,
                    "checkpoint_path": str(checkpoint_path),
                    "resolution": resolution,
                    "image_name": name,
                    "dice": metrics["dice"],
                    "iou": metrics["iou"],
                    "precision": metrics["precision"],
                    "recall": metrics["recall"],
                    "threshold": threshold,
                }
                per_image_metrics.append(record)

                pred_slice = preds_cpu[idx, 0].numpy().astype(np.uint8) if preds_cpu[idx].ndim == 3 else preds_cpu[idx].numpy().astype(np.uint8)
                gt_slice = masks_cpu[idx, 0].numpy().astype(np.uint8) if masks_cpu[idx].ndim == 3 else masks_cpu[idx].numpy().astype(np.uint8)
                prediction_samples.append(
                    {
                        "checkpoint": checkpoint_name,
                        "resolution": resolution,
                        "image_name": name,
                        "dice": metrics["dice"],
                        "pred_mask": pred_slice,
                        "gt_mask": gt_slice,
                    }
                )

    return per_image_metrics, prediction_samples


def run_resolution_evaluation(
    logs_root: Path,
    image_names: list[str],
    images_dir: Path,
    masks_dir: Path,
    output_root: Path,
    batch_size: int = 1,
    threshold: float = 0.5,
) -> tuple[pd.DataFrame, pd.DataFrame, dict]:
    logs_root = Path(logs_root).expanduser().resolve()
    images_dir = Path(images_dir).expanduser().resolve()
    masks_dir = Path(masks_dir).expanduser().resolve()
    output_root = Path(output_root).expanduser().resolve()
    output_root.mkdir(parents=True, exist_ok=True)

    if not image_names:
        raise ValueError("No image names provided for evaluation.")

    checkpoint_paths = sorted(logs_root.rglob("*.pth"))
    if not checkpoint_paths:
        raise FileNotFoundError(f"No checkpoints found under {logs_root}")

    per_image_records: list[dict[str, float]] = []
    per_checkpoint_stats: list[dict] = []
    predictions_cache: dict[tuple[int, str], dict] = {}

    for checkpoint_path in checkpoint_paths:
        resolution = parse_resolution_from_name(checkpoint_path)
        loader = build_resolution_loader(
            image_names=image_names,
            images_dir=images_dir,
            masks_dir=masks_dir,
            resize_to=resolution,
            batch_size=batch_size,
        )

        model = load_model_from_checkpoint(checkpoint_path, config["device"])
        per_image_metrics, prediction_samples = evaluate_checkpoint_simple(
            model=model,
            loader=loader,
            resolution=resolution,
            checkpoint_path=checkpoint_path,
            threshold=threshold,
            device=config["device"],
        )

        per_image_records.extend(per_image_metrics)

        if prediction_samples:
            dice_values = [sample["dice"] for sample in prediction_samples]
            avg_dice = float(np.mean(dice_values))
            best_sample = max(prediction_samples, key=lambda item: item["dice"])
            worst_sample = min(prediction_samples, key=lambda item: item["dice"])
        else:
            avg_dice = float("nan")
            best_sample = None
            worst_sample = None

        per_checkpoint_stats.append(
            {
                "checkpoint": checkpoint_path.name,
                "checkpoint_path": str(checkpoint_path),
                "resolution": resolution,
                "dice_mean": avg_dice,
                "dice_best": best_sample["dice"] if best_sample else float("nan"),
                "dice_best_image": best_sample["image_name"] if best_sample else None,
                "dice_worst": worst_sample["dice"] if worst_sample else float("nan"),
                "dice_worst_image": worst_sample["image_name"] if worst_sample else None,
                "sample_count": len(prediction_samples),
            }
        )

        predictions_cache[(resolution, checkpoint_path.name)] = {
            "average": avg_dice,
            "best": best_sample,
            "worst": worst_sample,
            "samples": prediction_samples,
        }

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    metrics_df = pd.DataFrame(per_image_records)
    metrics_csv = output_root / "metrics_per_image.csv"
    metrics_df.to_csv(metrics_csv, index=False)

    summary_df = pd.DataFrame(per_checkpoint_stats).sort_values(["resolution", "checkpoint"]).reset_index(drop=True)
    summary_csv = output_root / "metrics_summary.csv"
    summary_df.to_csv(summary_csv, index=False)

    print(f"Saved per-image metrics to {metrics_csv}")
    print(f"Saved checkpoint summary to {summary_csv}")

    return metrics_df, summary_df, predictions_cache



In [17]:
# ===============================
# Resolution Evaluation Configuration
# ===============================
resolution_logs_root = Path.cwd() / "Resolution_logs"

eval_image_names = list(val_imgs)
eval_images_dir = Path(images_directory)
eval_masks_dir = Path(masks_directory)

eval_batch_size = config.get("test_batch_size", 1)
dice_threshold = 0.5

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
output_root = Path.cwd() / "resolution_evaluation_outputs" / f"eval_{timestamp}"
output_root.mkdir(parents=True, exist_ok=True)

print(f"Logs root: {resolution_logs_root}")
print(f"Evaluation images directory: {eval_images_dir}")
print(f"Evaluation masks directory: {eval_masks_dir}")
print(f"Number of evaluation samples: {len(eval_image_names)}")
print(f"Output directory: {output_root}")


NameError: name 'val_imgs' is not defined

In [ ]:
# ===============================
# Run Resolution Evaluation
# ===============================
metrics_df, summary_df, predictions_cache = run_resolution_evaluation(
    logs_root=resolution_logs_root,
    image_names=eval_image_names,
    images_dir=eval_images_dir,
    masks_dir=eval_masks_dir,
    output_root=output_root,
    batch_size=eval_batch_size,
    threshold=dice_threshold,
)

summary_df


### Simple checkpoint evaluation workflow

1. Ensure `resolution_logs_root` points to the folder containing the `.pth` checkpoints (defaults to `Resolution_logs`).
2. The configuration cell reuses the validation image names (`val_imgs`) and the original `images_directory`/`masks_directory`; adjust these variables if your evaluation split differs.
3. Run the configuration cell, then execute **Run Resolution Evaluation**. For each checkpoint the loop:
   - builds a resized DataLoader at the inferred resolution,
   - loads the DeepLab Version 3 Plus model weights,
   - runs a straightforward validation loop over the loader, storing per-image predictions,
   - collects average, best, and worst Dice scores.
4. Inspect `summary_df` for per-checkpoint Dice statistics and open `metrics_per_image.csv` / `metrics_summary.csv` inside the timestamped `resolution_evaluation_outputs` folder for detailed metrics.
